In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


/var/folders/8j/rn97p49x1x78773bw6s8z9kw0000gs/T/ipykernel_7259/1423498497.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Knowledge representation basics 3:
### Expert Systems
### AIML as an example for creating knowledge bases for chatbots
### Forward Chaining

### Rule-Based Systems are often called "Expert systems"
 - Because they encode knowledge from a human expert, and use it to do reasoning.

<img src ="./figs/expert-system.png" style="float:left" >  

 - The collection of facts and rules embedded within an expert system is called the 'knowledge-base'
 - an inference engine works out which rules can fire, and then chooses between them
 - an 'explanation engine' may store the thread of reasoning

### AIML: An expert system for authoring chatbot knowledge bases
<img src ="./figs/chatbot.png" style="float:left" >

- Maintained by the ALICE foundation,now on version 2.0
- Some criticism as overly simple,  but can create powerful chatbots
- AIML is an XMl-based language


Last week you should have created a simple chatbot, and examined the preprocessing part of the system

This week we will focus on the simplest part of the knowledge base


### AIML is made of rules called "categories"

Each category contains some basic parts:
- A **pattern**:  the exact form of words that the chatbot recognises to trigger the rule, i.e.  the *condition*
- A **template**: the output of the rule, i.e. the *action*.

AIML is xml-based, so each part is started and finished with a tag pair in angled brackets e.g.  `<category> ..... </category>`

An *atomic* category is one where the pattern and template are simple text.   
- You can think of these as how you encode facts that you retrieve by querying with the appropriate question. As we'll see next week, both the pattern and the template can be richer. 

In [2]:
%%writefile "data/fruit.aiml"
<aiml>

<category>
    <pattern> DESCRIBE APPLES</pattern>
    <template> Apples are round fruit with smooth green skin</template>
</category>

<category>
  <pattern> IS MY FRUIT AN APPLE</pattern>
  <template> Is your fruit round and green and smooth?</template>
</category>
    
<category>
    <pattern>MY FRUIT IS ROUND AND GREEN AND SMOOTH</pattern>
    <template> Your fruit is an apple </template>
</category>
</aiml>

Overwriting data/fruit.aiml


In [3]:
import aiml

# Create the kernel and learn AIML files
myChatbot = aiml.Kernel()
myChatbot.learn("data/fruit.aiml")
myChatbot.verbose(True)
print( "Successfully loaded {} categories".format(myChatbot.numCategories()))

#type  bye to end the chat session
keepGoing = True
while keepGoing:
    nextInput = input("Enter your message >> ")
    if(nextInput == 'bye'):
        keepGoing = False
    else:
        print (myChatbot.respond(nextInput))

Loading data/fruit.aiml...done (0.00 seconds)
Successfully loaded 3 categories


### Forward Chaining you could definely talk about how this is like decision trees
<img style="float:right" src="figs/forward-chaining.png" width=600>  

<div style="float:left width=400">
<b>Data-driven</b> discovery of knowledge.
    <ol>
        <li> <b>Initialise</b> knowledge-set with any long-term facts</li>
        <li> <b>Add</b> assertions derived from observations</li>
     <li> <b>Find the match-set</b> of rules <br> whose conditions are met by the current short term knowledge</li>
        <li> <b>While</b> (match-set is not empty and decision is not reached):
            <ol> <li> <b>Rank</b> match-set using domain-specific rule precedence.</li>
                <li> <b>Select</b> highest ranked rule from the  match-set </li>
                <li><b>Apply</b> the chosen  rule, which might let you: <br> 
   assert new short-term knowledge (facts) directly  <br>
                    ask questions to get new knowledge from the user  </li>
                <li><b>Add</b> inferences to current set of shore term knowledge</li>
                <li> <b>Find</b> the new match-set. </li></ol>
<li> Output result</li>
 </ol>                                                </div>

### How does this work in AIML?
Using a **`<srai>`** tag-pair within a template lets you infer new things by asserting that two inputs are equivalent: IF A <=> B and B=>C  THEN   A=>C

For example, 
- IF (input *matches* "tell me about Apples")  
  AND ("tell me about Apples" *is equivalent to* "Describe Apples")  
  THEN output(response to "Describe Apples")
- or to put it another way it lets us say:  
  ("tell me about apples" == True) => ("Describe Apples" ==True)

I'm using the symbol => to stand for "implies"

In [ ]:
%%writefile "data/fruitlanguage.aiml"
<aiml>
<category>
    <pattern> TELL ME ABOUT APPLES</pattern>
    <template> <srai> DESCRIBE APPLES</srai> </template>
</category>
</aiml>

In [ ]:
myChatbot.learn("data/fruitlanguage.aiml")
nextInput = input("Enter your message >> ")
print (myChatbot.respond(nextInput))

### Notes
1. this example is long-term information about language:
 - that that two ways of asking a question are true.    
 - You could argue that it is 'more' generic in that it might apply to several different chatbots
2. This week's practical session will get you using &lt;srai&gt; tags
3. Next week we'll look at:
     - how to make knowledge about language more generic using wildcards
     - other ways of doing forward chaining in AIML by creating new variables to store short term knowlege